# 第10章 面向对象的设计原则

经典的23种设计模式在Python中并不怎么有效。在Python中更适用的是SOLID原则，即：



| 简称 | 全称                                                         | 解释                                                         |      |
| :--- | :----------------------------------------------------------- | :----------------------------------------------------------- | ---- |
| S    | [单一职责原则则](https://zh.m.wikipedia.org/wiki/单一功能原则) | 认为“[对象](https://zh.m.wikipedia.org/wiki/对象_(计算机科学))应该仅具有一种单一功能”的概念。 |      |
| O    | [开闭原则](https://zh.m.wikipedia.org/wiki/开闭原则)         | 认为“软件应该是对于扩展开放的，但是对于修改封闭的”的概念。   |      |
| L    | [里氏替换原则](https://zh.m.wikipedia.org/wiki/里氏替换原则) | 认为“程序中的对象应该是可以在不改变程序正确性的前提下被它的子类所替换的”的概念。参考[契约式设计](https://zh.m.wikipedia.org/wiki/契约式设计)。 |      |
| I    | [接口隔离原则](https://zh.m.wikipedia.org/wiki/接口隔离原则) | 认为“多个特定客户端接口要好于一个宽泛用途的接口”的概念。     |      |
| D    | [依赖反转原则](https://zh.m.wikipedia.org/wiki/依赖反转原则) | 认为一个方法应该遵从“依赖于抽象而不是一个实例”的概念。 [依赖注入](https://zh.m.wikipedia.org/wiki/依赖注入)是该原则的一种实现方式。 |      |



## 10.1 单一职责原则

SRP(single reponsibility principle)认为一个类应该仅有一个被修改的理由，即每个类都只承担一种职责。

单个类承担的职责越多，就意味着类越复杂，越难维护，如臭名昭著的：God Class。

让类变得更纯粹，一般来说会把大类拆成小类，当然你也可以拆成一个函数。

## 10.2 开放-关闭原则 

OCP(open-closed principle):类应该对扩展开放，对修改关闭。

比如说我们有个类，里面维护了一些条件判断语句，随着业务的发展，我们可能会新增新的判定条件。那我们就不得不修改原有的类了，而这就违背了OCP原则。那应该怎么做呢？

1. 通过继承改造代码：找到父类中不稳定会变动的内容，将这部分变化的封装成方法，子类通过继承重写这部分行为。
2. 通过依赖注入:将易变的部分通过初始化参数注入对象，最终利用多态特性达到OCP。
>对于依赖注入的部分，我们也可以写一个抽象类，来规范后续代码的实现。当然对于“鸭子类型”的Python这不是必须的，但这样做，可以结合“类型注解”，让代码更规范。
3. 数据驱动：与依赖注入类似，不过抽离的是纯粹的数据。代码更简洁，但是定制性略差。

## 10.3 里式替换原则

LSP(Liskov substitution principle)：所有子类对象都应该可以任意替代父类对象使用，且不会破坏程序原本的功能。

常见的违背方式有：

### 10.3.1 子类随意抛出异常

比如说我们有一个Use类，可以被停用。后来我们有个Admin类继承了User，但是在停用方法上rasie了一个异常，因为Admin不能被停用。而这就破话了LSP原则。

因为Admin类是后添加的，我们最初设计停用相关的代码时，是没有考虑用`try`捕获代码，所以会出现意料之外的错误。

解决方案是：定义专门的异常类，在父类User和子类Admin中添加抛出异常说明

### 10.3.2 子类随意调整参数与返回值

对于返回值类型，比如:
- User类方法foo返回`List[int]`
- Admind子类方法foo返回`Iterable[int]`

虽然绝大部分下子类都符合父类的功能，但是`len()`方法是个意外。所以这依然破坏了LSP原则。

但是如果将返回值调换一下：
- User类方法foo返回`Iterable[int]`
- Admind子类方法foo返回`List[int]`

就完全没有问题了！因为`List`是`Iterable`的子类，子类可以实现父类的所有特性。

---
子类方法的参数要与父类的完全一致，或者比父类的更加宽松。比如：
1. 子类方法可以有额外参数，但是要有默认值；
2. 子类方法同名参数类型可以完全实现父类的所有特性

> 总结：
>
> 子类不能抛出父类不知道的异常，不能返回与父类（或父类的派生类）不同的返回值

### 10.3.3 修改隐式约定

如果父类中有`@property`设定的约定限制，那么子类中虽然返回值和参数保持一致。但是依然不能破坏这些特性。

这种隐式破坏LSP，要比上面的更加难以察觉。

### 10.3.4 结论

如果破坏了LSP原则，那么Python多态的特性就不再可靠，一定要当心！

## 10.4 依赖倒置原则

DIP(dependency inversion principle)是一条与依赖关系相关的原则：高层模块不应该依赖低层模块，两者都应该依赖抽象。

简单来说，就是高层模块中的内容都是接口，而不是具体的实现。这样在`mock`测试的时候也更加方便。当然，也是视情况而定，要不要添加一层接口。


## 10.5 接口隔离原则

ISP(interface segregation principle)：一个接口所提供的方法应该刚好满足使用方的需求。

